<a href="https://colab.research.google.com/github/NicoCaz/Resumen_video_yt/blob/main/Resumen_de_video_de_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install pytube
!sudo apt-get remove -y youtube-dl
!sudo apt update && sudo apt install ffmpeg

In [ ]:
import os
from pytube import YouTube

#Cambia el link que quieras
LINK = "https://www.youtube.com/watch?v=vQVKG9nioAg&list=PLDnCB4ejta7XXjx7GSZ2NWQsBUxUttS78&index=1&t=1731s&ab_channel=PaulaBonifaziAquino"

# Obtener objeto YouTube
yt = YouTube(LINK)

# Obtener objeto Stream de audio
audio_stream = yt.streams.filter(only_audio=True).first()

# Descargar archivo de audio en una ubicación temporal
temp_file_path = audio_stream.download()

# Renombrar archivo a "audio.mp3" y mover a la carpeta actual
os.rename(temp_file_path, os.path.join(os.getcwd(), "audio.mp3"))


In [ ]:
import whisper
model = whisper.load_model("base")
result = model.transcribe("audio.mp3")

In [10]:
nombre_archivo = "texto_original.txt"

with open(nombre_archivo, "w") as archivo:
    archivo.write(result["text"])

# Cerrar el archivo
archivo.close()

In [ ]:
!pip install beautifulsoup4
!pip install urllib3
!pip install nltk
!python -m nltk.downloader punkt
!python -m nltk.downloader stopwords

In [25]:
import bs4 as bs  
import urllib.request  
import re
import nltk
from nltk import word_tokenize,sent_tokenize
import heapq  


In [26]:

# Tamaño del resumen, cuando mas grande el numero mas extenso el resumen
tamanio_resumen = 150

# Función para formatear el artículo
def formatear_articulo(texto):
    texto_formateado = re.sub('[^a-zA-Z]', ' ', texto)
    texto_formateado = re.sub(r'\s+', ' ', texto_formateado)
    return texto_formateado

# Función para calcular la frecuencia de palabras
def calcular_frecuencia_palabras(texto_formateado):
    palabras = nltk.word_tokenize(texto_formateado.lower())
    stopwords = nltk.corpus.stopwords.words('spanish')
    frecuencia_palabras = {word: palabras.count(word) for word in palabras if word not in stopwords}
    return frecuencia_palabras

# Función para obtener las oraciones
def obtener_oraciones(texto):
    return nltk.sent_tokenize(texto)

# Función para calcular la frecuencia de las oraciones
def calcular_frecuencia_oraciones(oraciones, frecuencia_palabras):
    max_oracion = {}
    for sent in oraciones:
        for word in nltk.word_tokenize(sent.lower()):
            if word in frecuencia_palabras.keys():
                if len(sent.split(' ')) < tamanio_resumen:
                    if sent not in max_oracion.keys():
                        max_oracion[sent] = frecuencia_palabras[word]
                    else:
                        max_oracion[sent] += frecuencia_palabras[word]
    return max_oracion

# Función para generar el resumen
def generar_resumen(texto):
    texto_formateado = formatear_articulo(texto)
    frecuencia_palabras = calcular_frecuencia_palabras(texto_formateado)
    oraciones = obtener_oraciones(texto)
    max_oracion = calcular_frecuencia_oraciones(oraciones, frecuencia_palabras)
    resumen_oracion = heapq.nlargest(7, max_oracion, key=max_oracion.get)
    resumen = ' '.join(resumen_oracion)
    return resumen


resumen = generar_resumen(result["text"])
print(resumen)


nombre_archivo = "salida.txt"

with open(nombre_archivo, "w") as archivo:
    archivo.write(resumen)

# Cerrar el archivo
archivo.close()





En la otra punta voy a tener un vas de direcciones relativamente chico, porque solamente tengo que direccionar a 6 valores distintos a 6 celdas, pero mi vas de datos va a ser de 16 bit porque tengo celdas de 16, y la que dijimos que era un vas de datos, era un conjunto de cables por donde circula en cada cable un bit, cada uno de 16 cables va a estar conectado a cada uno de estos bit de mi celda. Y una unidad de control que se encarga de manejar, administrar, usar, indicar qué cosas tienen que hacer la ALU, qué tienen que hacer los registros, si guardar un valor, si entregar ese valor, y cómo se direcciónas, cómo se selecciona el dispositivo en donde se va a guardar un determinado valor o recuperar un determinado valor. porque la unidad de decodificación le dijo mira, esta haciendo referencia a la variable X y a la variable Z y esa está en tal posición de memoria, etcétera, bueno, la unidad número tres, entonces va a ir a direccionar, va a poder acceder a la memoria a buscar esos datos